In [1]:
# Import relevant libraries and packages.
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing 
import warnings
from scipy import stats
import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#import sys

credentials = []
f = open('credentials.txt', 'r')
for line in f:
    #add line to credentials list (remove trailing whitespace characters)
    credentials.append(line.rstrip())

# create Spotify object with credentials
client_credentials_manager = SpotifyClientCredentials(credentials[0], credentials[1])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### may need to use np.select since there are two conditions:
    
https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column

another potentially useful stackoverflow: https://stackoverflow.com/questions/69669956/python-iterate-through-a-dataframe-and-check-if-a-value-from-a-list-exist-in-a

using isin()
https://www.geeksforgeeks.org/check-if-a-value-exists-in-a-dataframe-using-in-not-in-operator-in-python-pandas/

In [2]:
df6 = pd.read_csv("/Users/gregwelliver/Desktop/springboard_files/Capstone Two - Music/Data/Merged_Data7.csv", index_col =[0])

In [3]:
df6.head(30)

,track_name,track_genre,artist_name,track_popularity,popular_year,Top100,key,acousticness,danceability,duration_ms,...,loudness,modality,speechiness,tempo,time_signature,valence,second_artist,artist_country,collaboration,prev_Top100
0,The Real Slim Shady,hip-hop,Eminem,88.0,2000,Y,5.0,0.030200,0.949,284200.0,...,-4.244,0.0,0.0572,104.504,4.0,0.760,NaN,United States,N,N
1,Ms. Jackson,hip-hop,Outkast,82.0,2001,Y,4.0,0.143000,0.843,270507.0,...,-5.946,0.0,0.2690,94.948,4.0,0.613,NaN,NaN,N,N
2,Stan,hip-hop,Eminem,85.0,2001,Y,6.0,0.037100,0.780,404107.0,...,-4.325,0.0,0.2380,80.063,4.0,0.507,NaN,United States,N,N
3,Ride Wit Me,hip-hop,Nelly,75.0,2001,Y,7.0,0.061600,0.850,291782.0,...,-6.490,1.0,0.0478,101.875,4.0,0.722,NaN,United States,N,N
4,"So Fresh, So Clean",hip-hop,Outkast,72.0,0,N,5.0,0.028100,0.725,240027.0,...,-7.905,0.0,0.3320,166.028,3.0,0.915,NaN,NaN,N,N
5,The Way I Am,hip-hop,Eminem,75.0,0,N,4.0,0.144000,0.785,290427.0,...,-2.692,0.0,0.2650,87.248,4.0,0.315,NaN,United States,N,N
6,Country Grammar (Hot Shit),hip-hop,Nelly,69.0,2000,Y,2.0,0.006890,0.865,287000.0,...,-6.822,1.0,0.1080,162.831,4.0,0.565,NaN,United States,N,N
7,Kill You,hip-hop,Eminem,70.0,0,N,11.0,0.155000,0.897,264400.0,...,-8.268,1.0,0.3790,106.991,4.0,0.677,NaN,United States,N,N
8,Bitch Please II,hip-hop,Eminem,72.0,0,N,1.0,0.078300,0.918,288200.0,...,-2.890,1.0,0.1330,95.517,4.0,0.779,NaN,United States,N,N
9,What's Your Fantasy (Featuring Shawna),hip-hop,Ludacris,63.0,0,N,6.0,0.160000,0.915,275907.0,...,-3.186,0.0,0.2070,140.095,4.0,0.967,NaN,United States,Y,N


In [4]:
top100df = df6[df6['Top100'] == 'Y']
top100df.head(30)


,track_name,track_genre,artist_name,track_popularity,popular_year,Top100,key,acousticness,danceability,duration_ms,...,loudness,modality,speechiness,tempo,time_signature,valence,second_artist,artist_country,collaboration,prev_Top100
0,The Real Slim Shady,hip-hop,Eminem,88.0,2000,Y,5.0,0.03020,0.949,284200.0,...,-4.244,0.0,0.0572,104.504,4.0,0.760,NaN,United States,N,N
1,Ms. Jackson,hip-hop,Outkast,82.0,2001,Y,4.0,0.14300,0.843,270507.0,...,-5.946,0.0,0.2690,94.948,4.0,0.613,NaN,NaN,N,N
2,Stan,hip-hop,Eminem,85.0,2001,Y,6.0,0.03710,0.780,404107.0,...,-4.325,0.0,0.2380,80.063,4.0,0.507,NaN,United States,N,N
3,Ride Wit Me,hip-hop,Nelly,75.0,2001,Y,7.0,0.06160,0.850,291782.0,...,-6.490,1.0,0.0478,101.875,4.0,0.722,NaN,United States,N,N
6,Country Grammar (Hot Shit),hip-hop,Nelly,69.0,2000,Y,2.0,0.00689,0.865,287000.0,...,-6.822,1.0,0.1080,162.831,4.0,0.565,NaN,United States,N,N
11,Put It On Me,hip-hop,Ja Rule,64.0,2001,Y,1.0,0.00240,0.831,262827.0,...,-4.714,0.0,0.1650,93.140,4.0,0.791,NaN,United States,N,N
19,I Just Wanna Love U (Give It 2 Me),hip-hop,JAY-Z,60.0,2000,Y,4.0,0.30100,0.800,227867.0,...,-5.125,0.0,0.2400,98.631,4.0,0.801,NaN,United States,N,N
31,He Can't Love U,hip-hop,Jagged Edge,56.0,2000,Y,8.0,0.11200,0.721,244053.0,...,-3.972,0.0,0.2060,126.279,4.0,0.508,NaN,United States,N,N
34,Shake Ya Ass,hip-hop,Mystikal,57.0,2000,Y,7.0,0.06260,0.914,256973.0,...,-5.658,1.0,0.3200,98.054,4.0,0.666,NaN,United States,N,N
36,Promise,hip-hop,Jagged Edge,55.0,2001,Y,5.0,0.13600,0.751,246720.0,...,-5.431,0.0,0.0872,128.002,4.0,0.646,NaN,United States,N,N


In [5]:
top100df[top100df['artist_name'] == 'Nelly']

,track_name,track_genre,artist_name,track_popularity,popular_year,Top100,key,acousticness,danceability,duration_ms,...,loudness,modality,speechiness,tempo,time_signature,valence,second_artist,artist_country,collaboration,prev_Top100
3,Ride Wit Me,hip-hop,Nelly,75.0,2001,Y,7.0,0.06160,0.850,291782.0,...,-6.490,1.0,0.0478,101.875,4.0,0.722,NaN,United States,N,N
6,Country Grammar (Hot Shit),hip-hop,Nelly,69.0,2000,Y,2.0,0.00689,0.865,287000.0,...,-6.822,1.0,0.1080,162.831,4.0,0.565,NaN,United States,N,N
22821,Hot In Herre,hip-hop,Nelly,75.0,2002,Y,11.0,0.20600,0.956,228240.0,...,-4.753,0.0,0.1200,107.075,4.0,0.912,NaN,United States,N,N
22822,Dilemma,hip-hop,Nelly,77.0,2002,Y,2.0,0.22700,0.727,289160.0,...,-8.074,0.0,0.1400,168.189,4.0,0.607,NaN,United States,N,N
22835,Air Force Ones,hip-hop,Nelly,62.0,2003,Y,4.0,0.08470,0.784,304000.0,...,-9.740,0.0,0.3170,164.062,4.0,0.618,NaN,United States,N,N
45795,Over And Over,hip-hop,Nelly,61.0,2005,Y,7.0,0.00650,0.644,253933.0,...,-10.020,1.0,0.0803,169.799,4.0,0.464,NaN,United States,N,N
45877,My Place,hip-hop,Nelly,58.0,2004,Y,1.0,0.04980,0.600,336507.0,...,-4.237,0.0,0.0423,163.153,4.0,0.642,NaN,United States,N,N
46814,Grillz,hip-hop,Nelly,57.0,2006,Y,4.0,0.03580,0.867,271160.0,...,-7.737,0.0,0.2410,82.996,4.0,0.840,NaN,United States,N,N
114642,Just A Dream,hip-hop,Nelly,74.0,2010,Y,1.0,0.04210,0.531,237800.0,...,-6.161,1.0,0.0305,89.917,4.0,0.103,NaN,United States,N,N


In [6]:
top100df.Top100.value_counts()

Y    1863
Name: Top100, dtype: int64

In [7]:
top100df['artist_name']

0                Eminem
1               Outkast
2                Eminem
3                 Nelly
6                 Nelly
              ...      
238455    Ariana Grande
238687          ROSALÍA
249131         Dua Lipa
249545      Demi Lovato
249637         DJ Snake
Name: artist_name, Length: 1863, dtype: object

dict = dict([(i,[x]) for i,x in zip(top100df['artist_name'], top100df['popular_year'])])

In [8]:
print(dict)

<class 'dict'>


In [10]:
#top100df.set_index('artist_name').T.to_dict('popular_year')

In [11]:
df6.prev_Top100.value_counts()

N    255096
Name: prev_Top100, dtype: int64

In [12]:
new = df6["artist_name"].isin(top100df["artist_name"])
new.value_counts()

False    205033
True      50063
Name: artist_name, dtype: int64

In [15]:
for row in df6['artist_name']:
    if df6["artist_name"].isin(top100df["artist_name"]):
        print('yes')
    else:
        print('no')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

for row in df6['artist_name']:
    df6["artist_name"].isin(top100df["artist_name"])

In [16]:
for row in df6[df6['artist_name']]:
    if df6[df6['artist_name']] in top100df[top100df['artist_name']]:
        print('yes')
    else:
        print('not feat')

KeyError: "None of [Index(['Eminem', 'Outkast', 'Eminem', 'Nelly', 'Outkast', 'Eminem', 'Nelly',\n       'Eminem', 'Eminem', 'Ludacris',\n       ...\n       'Lynyrd Skynyrd', 'Brittany Howard', 'The Bones of J.R. Jones',\n       'The Who', 'Zayde Wølf', 'Samantha Fish', 'Black Pistol Fire',\n       'Badfinger', 'Dinosaur Jr.', 'Joanne Shaw Taylor'],\n      dtype='object', length=255096)] are in the [columns]"

In [17]:
for row in df6['artist_name']:
    if df6['artist_name'] in top100df['artist_name']:
        print('yes')
    else:
        print('no')

TypeError: unhashable type: 'Series'